In [1]:
from ipywidgets import FileUpload, Output
import ipywidgets as widgets
from IPython.display import display, Javascript,  HTML
import sys
from io import StringIO
import pandas as pd
import plotly.express as px
import numpy as np 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.signal import find_peaks

out = Output()
upload = FileUpload(accept='.txt', multiple=False)

upload 

FileUpload(value={}, accept='.txt', description='Upload')

In [2]:
surfaceInput = widgets.BoundedFloatText(
    value=0.1,
    min=0,
    max=10.0,
    step=0.1,
    description='Surface:',
    disabled=False
)
display(surfaceInput)

BoundedFloatText(value=0.1, description='Surface:', max=10.0, step=0.1)

In [3]:
# name = input('Please upload a file and a surface value. Press Enter when done.  \n')
def on_button_clicked(b):
    with out:
        out.clear_output()
        if(bool(upload.value)):  

            for uploaded_filename in upload.value:
                content = upload.value[uploaded_filename]['content']   
                print(uploaded_filename)
            upload.value.clear()
            upload._counter = 0

            s = "" 
            for line in content.split(b'\n'):
                if "Source;Delais(s);Mesure ;Temps(s);Mesure/Source;Source/Mesure;Puissance" in line.decode('latin-1'):
                    s = ""
                s +=line.decode('latin-1')

            df = pd.read_csv(StringIO(s), sep=";")
            df['Jsc'] = df['Mesure ']*1000/surfaceInput.value

            fig = make_subplots(rows=1, cols=2)

            fig1 = px.line(df,x="Source", y="Jsc")
            fig2 = px.line(df,x="Source", y="Puissance")

            fig.add_trace(fig1['data'][0], row=1, col=1)
            fig.add_trace(fig2['data'][0], row=1, col=2)

            peaks, _ = find_peaks(df.Puissance, height=0)

            fig.add_trace(go.Scatter(x=[df.Source[peaks[0]]],y=[df.Puissance[peaks[0]]],mode='markers'),row =1 , col =2)

            fig.update_layout(
                showlegend=False,
                autosize=False,
                width=1500,
                height=700,
                margin=dict(
                    l=50,
                    r=50,
                    b=100,
                    t=100,
                    pad=3
                )
            )
            # Update xaxis properties
            fig.update_xaxes(title_text="Source", row=1, col=1)
            fig.update_xaxes(title_text="Source", row=1, col=2)

            # Update yaxis properties
            fig.update_yaxes(title_text="Jsc", row=1, col=1)
            fig.update_yaxes(title_text="Puissance", row=1, col=2)

            # Update title and height
            # fig.update_layout(title_text="Plots")
            fig.show()

            print("Jsco is: " + str(np.interp(0, df.Source, df.Jsc)))
            print("Voc is: " + str(np.interp(0, df.Jsc, df.Source)))
            print("Minimum puissance: " + str(min(df.Puissance)))
            print("Maximum puissance: " + str(df.Puissance[peaks[0]]) + " at " + str(df.Source[peaks[0]]) + " V and " + str(df['Mesure '][peaks[0]]) + " A."  )
        else: 
            print("Empty file")

In [4]:
button = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check', # (FontAwesome names without the `fa-` prefix)
)
button.on_click(on_button_clicked)
display(button)
out

Button(description='Plot', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()

In [5]:

# %matplotlib inline

# from matplotlib.pyplot import *

# button = widgets.Button(description="Button")
# # out = Output()

# # def on_button_clicked(b):
# #     with out:
# #         out.clear_output()
# #         print("Asdf")
# #         plot([1,2],[2,1])
# #         show()

# button.on_click(on_button_clicked)

# display(button)
# out